In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import warnings

from sklearn.datasets import fetch_olivetti_faces
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics import adjusted_rand_score
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import homogeneity_completeness_v_measure
from scipy.spatial import distance

from AdaptiveKLLE import *

from sklearn.datasets import load_iris

In [2]:
import random
np.random.seed(0)
random.seed(0)


n_osservazioni = 1700
n_variabili = 17
R, r = 2, 1  
a, b = 0.1, 0.1  
r_sfera = 1  


theta = np.linspace(0, 2*np.pi, n_osservazioni)
phi = np.linspace(0, 2*np.pi, n_osservazioni)
x_toro = (R + r * np.cos(theta)) * np.cos(phi)
y_toro = (R + r * np.cos(theta)) * np.sin(phi)
z_toro = r * np.sin(theta)
toro = np.column_stack((x_toro, y_toro, z_toro))
toro = np.hstack([toro]+[np.random.normal(0, 0.05, (n_osservazioni, 3))] + [np.random.normal(0, 0.05, (n_osservazioni, n_variabili - 3))])

t = np.linspace(0, 20*np.pi, n_osservazioni)
x_spirale = a * t * np.cos(t)
y_spirale = a * t * np.sin(t)
z_spirale = b * t
spirale_3d = np.column_stack((x_spirale, y_spirale, z_spirale))
spirale_3d = np.hstack([spirale_3d]+[np.random.normal(0, 0.05, (n_osservazioni, 3))] + [np.random.normal(0, 0.05, (n_osservazioni, n_variabili - 3))])

np.random.seed(0)
phi = np.random.uniform(0, np.pi, n_osservazioni)
theta = np.random.uniform(0, 2*np.pi, n_osservazioni)
x_sfera = r_sfera * np.sin(phi) * np.cos(theta)
y_sfera = r_sfera * np.sin(phi) * np.sin(theta)
z_sfera = r_sfera * np.cos(phi)
sfera = np.column_stack((x_sfera, y_sfera, z_sfera))
sfera = np.hstack([sfera]+[np.random.normal(0, 0.05, (n_osservazioni, 3))] + [np.random.normal(0, 0.05, (n_osservazioni, n_variabili - 3))])

dataframes_complessi = [pd.DataFrame(toro), pd.DataFrame(spirale_3d), pd.DataFrame(sfera)]
X = pd.concat(dataframes_complessi, ignore_index=True).values.astype(float)
X.shape

y = np.repeat(np.arange(0, 3), n_osservazioni)

df_ = pd.DataFrame(np.column_stack((y, X))).sample(frac = 1, random_state = 0)
X = df_.iloc[:, 1:].values
y = df_.iloc[:, 0].values

In [3]:
def return_ids_kstar_binomial(data, embeddings, initial_id=None, Dthr=6.67, r='opt', n_iter = 10):
    if initial_id is None:
        data.compute_id_2NN(algorithm='base')
    else:
        data.compute_distances()
        data.set_id(initial_id)

    ids = np.zeros(n_iter)
    ids_err = np.zeros(n_iter)
    kstars = np.zeros((n_iter, data.N), dtype=int)
    log_likelihoods = np.zeros(n_iter)
    ks_stats = np.zeros(n_iter)
    p_values = np.zeros(n_iter)

    for i in range(n_iter):

      data.compute_kstar(Dthr)

      r_eff = min(0.95,0.2032**(1./data.intrinsic_dim)) if r == 'opt' else r
      
      rk = np.array([dd[data.kstar[j]] for j, dd in enumerate(data.distances)])
      rn = rk * r_eff
      n = np.sum([dd < rn[j] for j, dd in enumerate(data.distances)], axis=1)
      
      id = np.log((n.mean() - 1) / (data.kstar.mean() - 1)) / np.log(r_eff)
      
      id_err = ut._compute_binomial_cramerrao(id, data.kstar-1, r_eff, data.N)
      
      log_lik = ut.binomial_loglik(id, data.kstar - 1, n - 1, r_eff)
      
      n_model = rng.binomial(data.kstar-1, r_eff**id, size=len(n))
      ks, pv = ks_2samp(n-1, n_model)
      
      data.set_id(id)

      ids[i] = id
      ids_err[i] = id_err
      kstars[i] = data.kstar
      log_likelihoods[i] = log_lik
      ks_stats[i] = ks
      p_values[i] = pv

    data.intrinsic_dim = id
    data.intrinsic_dim_err = id_err
    data.intrinsic_dim_scale = 0.5 * (rn.mean() + rk.mean())

    return ids, kstars[(n_iter - 1), :]


def find_single_k_neighs(embeddings, index, k):
    target_embedding = embeddings[index]
    all_distances = np.array([distance.euclidean(target_embedding, emb) for emb in embeddings])

    nearest_indices = np.argsort(all_distances)[1:k+1]  

    return nearest_indices.tolist()

def find_adaptive_test(id_, X_test):
    data = Data(X_test)
    data.compute_id_2NN(algorithm='base')
    kstars_test = np.zeros(X_test.shape[0], dtype=int)
    Dthr = 6.67
    data.compute_kstar(Dthr)


    r_eff = min(0.95,0.2032**(1./id_)) if r == 'opt' else r

    rk = np.array([dd[data.kstar[j]] for j, dd in enumerate(data.distances)])
    rn = rk * r_eff
    n = np.sum([dd < rn[j] for j, dd in enumerate(data.distances)], axis=1)

    id = np.log((n.mean() - 1) / (data.kstar.mean() - 1)) / np.log(r_eff)

    id_err = ut._compute_binomial_cramerrao(id, data.kstar-1, r_eff, data.N)

    log_lik = ut.binomial_loglik(id, data.kstar - 1, n - 1, r_eff)

    n_model = rng.binomial(data.kstar-1, r_eff**id, size=len(n))
    ks, pv = ks_2samp(n-1, n_model)
    return data.kstar

In [4]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import warnings
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings("ignore")

folds = 3
n_iter = 15
r = 'opt'
random_state = 0


accuracy_llestar = []
accuracy_lle_no_hyper = []
accuracy_lle_comp = []
accuracy_lle_same = []
ids_ = []
num_kstars = []


kf = KFold(n_splits=folds, shuffle=True, random_state=random_state)


for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"\nFold {fold_idx + 1}/{folds}")
    

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    print(f"  Train size: {len(train_idx)}, Test size: {len(test_idx)}")
    

    print("  Calculate K-star LLE...")
    k_star_lle = K_starLLE(X=X_train, initial_id=None, n_iter=n_iter)
    Y_kstar, W_train, kstars = k_star_lle.calculate_embedding(initial_id=None, Dthr=6.67, r='opt')
    

    id_ = k_star_lle.return_ids_kstar_binomial(verbose=False)[0][n_iter-1]
    ids_.append(id_)
    num_kstars.append(int(np.round(np.median(kstars))))
    
    print(f"  ID optimal: {id_:.2f}, K-star median: {int(np.round(np.median(kstars)))}")
    

    lr = LogisticRegression(n_jobs=-1, random_state=0, penalty = None)
    lr.fit(Y_kstar, y_train)
    

    W = np.zeros((X_test.shape[0], X_train.shape[0]))
    
    for i in tqdm(range(X_test.shape[0])):

        new_data = np.concatenate((X_test[i, :].reshape(1, -1), X_train))
        

        data = Data(new_data)
        data.set_id(id_)
        data.compute_id_2NN(algorithm='base')
        data.compute_kstar(Dthr=6.67)
        k_s = data.kstar
        
        
        nns = find_single_k_neighs(new_data, 0, k_s[0])
        nns = np.array(nns) - 1 
        
        Z = X_train[nns] - X_test[i]  
        C = np.dot(Z, Z.T)  
        
        trace = np.trace(C)
        if trace > 0:
            R = 1e-3 * trace
        else:
            R = 1e-3
        C.flat[:: len(nns) + 1] += R    
    
        w = solve(C, np.ones(len(nns)), assume_a="pos")  
        W[i, nns] = w / np.sum(w)
    
    Y_kstar_test = np.dot(W, Y_kstar)
    
    preds_lr = lr.predict(Y_kstar_test)
    
    acc = accuracy_score(y_test, preds_lr)
    
    accuracy_llestar.append(acc)
    
    print(f"  Accuracy: {acc:.4f}")



Fold 1/3
  Train size: 3400, Test size: 1700
  Calculate K-star LLE...
iteration  0
id  [8.63]
iteration  1
id  [5.65]
iteration  2
id  [4.48]
iteration  3
id  [3.71]
iteration  4
id  [3.15]
iteration  5
id  [2.8]
iteration  6
id  [2.69]
iteration  7
id  [2.69]
iteration  8
id  [2.69]
iteration  9
id  [2.69]
iteration  10
id  [2.69]
iteration  11
id  [2.69]
iteration  12
id  [2.69]
iteration  13
id  [2.69]
iteration  14
id  [2.69]
  ID optimal: 2.69, K-star median: 48


/Users/federico.ravenda/anaconda3/envs/LLE/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
100%|███████████████████████████████████████| 1700/1700 [01:34<00:00, 18.07it/s]


  Accuracy: 0.8853

Fold 2/3
  Train size: 3400, Test size: 1700
  Calculate K-star LLE...
iteration  0
id  [8.71]
iteration  1
id  [5.71]
iteration  2
id  [4.57]
iteration  3
id  [3.76]
iteration  4
id  [3.16]
iteration  5
id  [2.79]
iteration  6
id  [2.68]
iteration  7
id  [2.66]
iteration  8
id  [2.66]
iteration  9
id  [2.66]
iteration  10
id  [2.66]
iteration  11
id  [2.66]
iteration  12
id  [2.66]
iteration  13
id  [2.66]
iteration  14
id  [2.66]
  ID optimal: 2.66, K-star median: 48


/Users/federico.ravenda/anaconda3/envs/LLE/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
100%|███████████████████████████████████████| 1700/1700 [01:35<00:00, 17.82it/s]


  Accuracy: 0.9547

Fold 3/3
  Train size: 3400, Test size: 1700
  Calculate K-star LLE...
iteration  0
id  [9.02]
iteration  1
id  [5.72]
iteration  2
id  [4.49]
iteration  3
id  [3.67]
iteration  4
id  [3.03]
iteration  5
id  [2.7]
iteration  6
id  [2.62]
iteration  7
id  [2.63]
iteration  8
id  [2.63]
iteration  9
id  [2.63]
iteration  10
id  [2.63]
iteration  11
id  [2.63]
iteration  12
id  [2.63]
iteration  13
id  [2.63]
iteration  14
id  [2.63]
  ID optimal: 2.63, K-star median: 49


/Users/federico.ravenda/anaconda3/envs/LLE/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
100%|███████████████████████████████████████| 1700/1700 [01:34<00:00, 17.96it/s]

  Accuracy: 0.9300


In [5]:
print(np.mean(accuracy_llestar))

0.9233333333333333


In [8]:
import warnings


warnings.filterwarnings("ignore")


folds = 3
r = 'opt'
random_state = 0


predictions_no_hyper = []
predictions_same = []


kf = KFold(n_splits=folds, shuffle=True, random_state=random_state)


for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X)):

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
        
    ###no hyper
    lle = LocallyLinearEmbedding(random_state=0)
    X_train_lle = lle.fit_transform(X_train)
    
    lr_lle = LogisticRegression(n_jobs=-1, random_state=0, penalty = None)
    lr_lle.fit(X_train_lle, y_train)
    
    X_test_lle = lle.transform(X_test)
    
    preds_lr_lle = lr_lle.predict(X_test_lle)
    
    predictions_no_hyper.append(preds_lr_lle)
    accuracy_lle_no_hyper.append(accuracy_score(y_test, preds_lr_lle))
    

In [9]:
print(np.mean(accuracy_lle_no_hyper))

0.8168627450980392
